In [ ]:
#Install required packages
!pip install transformers torch scikit-learn

In [ ]:
#Import necessary libraries
import os
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from sklearn.preprocessing import LabelEncoder
import re

In [ ]:
#Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
#Load the model and tokenizer
model_path = "/content/business-sentiment-model"  #Replace with your model folder
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForSequenceClassification.from_pretrained(model_path)
model.to(device)
model.eval()

In [ ]:
#Label encoder for decoding predictions
labels = ['Negative', 'Neutral', 'Positive']  # or load from disk if custom
label_encoder = LabelEncoder()
label_encoder.fit(labels)

In [ ]:
#Preprocessing function
def clean_text(text):
    text = re.sub(r'\s+', ' ', text)  # Remove excess whitespace
    text = re.sub(r'http\S+', '', text)  # Remove URLs
    text = re.sub(r'[^A-Za-z0-9.,!?\'" ]+', '', text)  # Remove special chars
    return text.strip()

In [ ]:
#Inference function
def predict_sentiment(text):
    text = clean_text(text)
    inputs = tokenizer(text, return_tensors='pt', truncation=True, padding=True, max_length=512)
    inputs = {k: v.to(device) for k, v in inputs.items()}
    with torch.no_grad():
        outputs = model(**inputs)
    probs = torch.nn.functional.softmax(outputs.logits, dim=1)
    pred = torch.argmax(probs, dim=1)
    label = label_encoder.inverse_transform(pred.cpu().numpy())[0]
    return label, probs.cpu().numpy()

In [ ]:
#Read .txt files and predict
def analyze_directory(folder_path):
    results = []
    for file in os.listdir(folder_path):
        if file.endswith(".txt"):
            file_path = os.path.join(folder_path, file)
            with open(file_path, 'r', encoding='utf-8') as f:
                text = f.read()
            label, probs = predict_sentiment(text)
            results.append((file, label, probs))
    return results

In [ ]:
#Run on folder of .txt files
folder_path = "/content/txt_files"  #Upload your .txt files here
sentiment_results = analyze_directory(folder_path)

In [ ]:
#Print results
for file, label, probs in sentiment_results:
    print(f"{file} → Sentiment: {label} | Probabilities: {probs}")